A proof of concept showing using a genetic algorithm with our environment.
It is similar to https://github.com/DEAP/deap/blob/a0b78956e28387785e3bb6e2b4b1f1b32c2b3883/examples/ga/onemax_short.py

In [1]:
# Run this cell if you're using colab. Otherwise, skip it.

!git clone https://github.com/platers/meta-transfer-learning.git

import os
os.chdir('meta-transfer-learning')

!pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.9.0.dev0-cp36-cp36m-manylinux1_x86_64.whl

!pip install -r requirements.txt
!pip install ran

fatal: destination path 'meta-transfer-learning' already exists and is not an empty directory.
  Using cached https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.9.0.dev0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: ray 0.9.0.dev0
    Uninstalling ray-0.9.0.dev0:
      Successfully uninstalled ray-0.9.0.dev0
ERROR: Could not find a version that satisfies the requirement ran (from versions: none)
ERROR: No matching distribution found for ran


In [0]:
import array
import random

import numpy as np
from typing import Dict

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import gym

from environments import SimpleEnv
import importlib
importlib.reload(SimpleEnv)
from environments.SimpleEnv import SimpleEnv #, TODO: add more environments

from ray.rllib.agents import ppo
from ray import tune
from ray.rllib.policy.policy import Policy
from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.agents.callbacks import DefaultCallbacks
import ray

import time
start = time.process_time()

from ray import tune


In [0]:
# some settings that we can tweak:
config_evolution={
        "MAX_STEP_COUNT": 1, # number of steps in each round of DEAP evolution 
        "POPULATION_SIZE": 1, # population in each round of DEAP evolution 
        "N_GEN": 1 # number of rounds of DEAP evolution This has the most effect
        }
N_RL_TRANING = 1 # number of rounds of RL training
TRAIN_BATCH_SIZE = 1 #Batch size for RL training
LEARNING_RATE = 0.001 #learning rate for RL training
EVAL_METHOD = "OPTIMAL" # RL or OPTIMAL
TUNING_EVOLUTION = True

In [4]:
ray.shutdown()
ray.init()
n_agents = 3
n_var = 2
training_envs = [
(SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'true_reward_weights': [1, 0],
    'max_step_count': config_evolution["MAX_STEP_COUNT"],
}),
(SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'true_reward_weights': [0, 1],
    'max_step_count': config_evolution["MAX_STEP_COUNT"],
})]
test_env = (SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'true_reward_weights': [1, 1],
    'max_step_count': config_evolution["MAX_STEP_COUNT"],
})

# creator.create('FitnessMax', base.Fitness, weights=(1.0, ))
# creator.create('Individual', array.array, typecode='d',
#                fitness=creator.FitnessMax)

# toolbox = base.Toolbox()

# toolbox.register('attr', random.uniform, -1, 1)
# toolbox.register('individual', tools.initRepeat, creator.Individual,
#                  toolbox.attr, n_agents * n_var)
# toolbox.register('population', tools.initRepeat, list,
#                  toolbox.individual)


2020-05-08 00:56:49,577	INFO resource_spec.py:212 -- Starting Ray with 7.18 GiB memory available for workers and up to 3.59 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-08 00:56:49,990	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


In [0]:
# Directly generate "optimal action" suggested by the reward function created by Evolution
# reward_weights is created from env(config=env_config)
def get_optimal_action(reward_weights, env_config):
    n_vars = env_config["n_vars"]
    max_act = 5
    
    reward_scale_factor = np.array([1]*n_vars + [2]*(len(reward_weights)-n_vars))
    scaled_reward_weights = reward_weights * reward_scale_factor
    best_act = np.argmax(scaled_reward_weights)
    action = np.eye(len(reward_weights))[best_act] * max_act
    action = np.reshape(action, [n_agents, n_vars])
    return action 

# If we do not want to optimize RL but just want to optimize reward function using Evolution function
# then here we just do the "optimal action" suggested by the generated reward function
def evaluate_individual_env_optimal_act(individual, environment_fn, env_config):
    env_config['reward_weights'] = np.array([individual for i in range(n_agents)])
    env = environment_fn(config=env_config)
    
    ave_true_rewards = 0
    obs = env.reset()
    ave_reward = 0
    for _ in range(env_config["max_step_count"]):
        actions = {i: get_optimal_action(env.reward_weights[i], env_config) for i in range(n_agents)}
        obs, reward, _, _ = env.step(actions)
        reward = np.array([reward[i] for i in range(len(reward))])
        ave_reward += reward
        ave_true_rewards += env.last_true_reward
        
    ave_true_rewards /= env_config["max_step_count"]
    ave_reward /= env_config["max_step_count"]
    
    return np.mean(ave_true_rewards)

In [0]:
# train RL agent with given setup
def evaluate_individual_env_rl(individual, environment, env_config):
    """Runs the environment. All agents have the same policy.
  It returns the total true reward as the fitness.
  """
    #Select random individuals from pop and create the reward weights
    pop = np.array([individual for i in range(n_agents)])
    reward_weights = pop
    env_config['reward_weights'] = reward_weights
    #env is only to get action space and observation space
    env = environment(config=env_config)
    class MyCallbacks(DefaultCallbacks):
        #Callback functions to keep track of true reward while training
        def on_episode_start(self, worker: RolloutWorker, base_env: BaseEnv,
                         policies: Dict[str, Policy],
                         episode: MultiAgentEpisode, **kwargs):
            episode.user_data["true_rewards"] = np.zeros(n_agents)

        def on_episode_step(self, worker: RolloutWorker, base_env: BaseEnv,
                        episode: MultiAgentEpisode, **kwargs):
            env = base_env
            true_reward = env.env_states[0].env.last_true_reward
            episode.user_data["true_rewards"] += true_reward

        def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: MultiAgentEpisode,
                       **kwargs):
            true_reward = episode.user_data["true_rewards"]
            for i, r in enumerate(true_reward):
                episode.custom_metrics["true_reward_agent_" + str(i)] = r
    
    # settings for the RL agent trainer     
    config={
        "train_batch_size": TRAIN_BATCH_SIZE,
        "lr": LEARNING_RATE,
        "sgd_minibatch_size": TRAIN_BATCH_SIZE,
        "multiagent": {
            "policies": {
            },
            "policy_mapping_fn":  #all agents share a policy
                lambda agent_id:
                    'agent'
        },
        "model": {"fcnet_hiddens": []},
        'env_config': env_config,
        "callbacks": MyCallbacks,
    }
    config['multiagent']['policies']['agent'] = (None, env.observation_space, env.action_space, {})
    metrics = None
    while True:
        trainer = ppo.PPOTrainer(env=environment, config=config)
        true_reward_mean = 0
        for i in range(N_RL_TRANING):
            true_reward_mean = 0
            #Train the RL agent
            metrics = trainer.train()  # distributed training step
            print("episode_reward_mean", metrics["episode_reward_mean"])
            if metrics["episode_reward_mean"] < 0 and i > 2:
                break
        #Train agent until it does well
        if metrics["episode_reward_mean"] > 0:
            break
    print("episode_reward_mean", metrics["episode_reward_mean"])
    for i in range(n_agents):
        true_reward_mean += metrics['custom_metrics']['true_reward_agent_' + str(i) + '_mean']
    true_reward_mean /= n_agents
    print('Evaluated', individual, 'Fitness', true_reward_mean)
    return true_reward_mean

In [0]:
if EVAL_METHOD == "RL":
    evaluate_individual_env = evaluate_individual_env_rl
else:
    evaluate_individual_env = evaluate_individual_env_optimal_act
        
def evaluate_individual(individual):
    """Runs all environments. 
  returns the average true reward over all environments as the fitness.
  """
    
    fitness = 0
    for env, config in training_envs:
        fitness += evaluate_individual_env(individual, env, config)
    fitness /= len(training_envs)
    return (fitness, )




In [8]:


#function usin tuning
def train_reward_function(config):
  creator.create('FitnessMax', base.Fitness, weights=(1.0, ))
  creator.create('Individual', array.array, typecode='d',
                fitness=creator.FitnessMax)

  toolbox = base.Toolbox()

  toolbox.register('attr', random.uniform, -1, 1)
  toolbox.register('individual', tools.initRepeat, creator.Individual,
                  toolbox.attr, n_agents * n_var)
  toolbox.register('population', tools.initRepeat, list,
                  toolbox.individual)
    # some setup for Deap
  toolbox.register('evaluate', evaluate_individual)
  toolbox.register('mate', tools.cxTwoPoint)
  toolbox.register('mutate', tools.mutFlipBit, indpb=0.05) # add more noise
  toolbox.register('select', tools.selTournament, tournsize=3)

  # pop is a list individual reward function's weights 
  pop = toolbox.population(n=config["POPULATION_SIZE"])
  # the 10 best individual reward function (could even befround the first round)
  hof = tools.HallOfFame(10)

  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register('avg', np.mean)
  stats.register('std', np.std)
  stats.register('min', np.min)
  stats.register('max', np.max)

  
  pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=config["N_GEN"], 
                                    stats=stats, halloffame=hof, verbose=True)
  best_individual = hof[0]
  test_reward = evaluate_individual_env(best_individual, test_env[0], test_env[1])
  print("---test_reward:", test_reward)
  tune.track.log(mean_accuracy=test_reward)

if TUNING_EVOLUTION:
  #configurations
  analysis = tune.run(
      train_reward_function, config={
          "N_GEN": tune.grid_search([1, 4]), 
          "POPULATION_SIZE": tune.grid_search([1])
          })
  #print best config
  print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))
else:
  train_reward_function(config_evolution)


# print ('pop', pop)

2020-05-08 00:56:50,766	WARNING tune.py:316 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,N_GEN,POPULATION_SIZE
train_reward_function_00000,RUNNING,,1,1
train_reward_function_00001,PENDING,,4,1
train_reward_function_00002,PENDING,,1,3
train_reward_function_00003,PENDING,,4,3


2020-05-08 00:56:53,381	WARNING worker.py:1094 -- The dashboard on node 5516454f8e45 failed with the following error:
Traceback (most recent call last):
  File "/usr/lib/python3.6/asyncio/base_events.py", line 1062, in create_server
    sock.bind(sa)
OSError: [Errno 99] Cannot assign requested address

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ray/dashboard/dashboard.py", line 1221, in <module>
    dashboard.run()
  File "/usr/local/lib/python3.6/dist-packages/ray/dashboard/dashboard.py", line 595, in run
    aiohttp.web.run_app(self.app, host=self.host, port=self.port)
  File "/usr/local/lib/python3.6/dist-packages/aiohttp/web.py", line 433, in run_app
    reuse_port=reuse_port))
  File "/usr/lib/python3.6/asyncio/base_events.py", line 484, in run_until_complete
    return future.result()
  File "/usr/local/lib/python3.6/dist-packages/aiohttp/web.py", line 359, in _run_app
    

(pid=2987) 2020-05-08 00:56:57,224	INFO trainable.py:217 -- Getting current IP.
Result for train_reward_function_00000:
  date: 2020-05-08_00-56-57
  done: false
  experiment_id: 3c99477c79ab40cd9bab8a80d3f4eb0e
  experiment_tag: 0_N_GEN=1,POPULATION_SIZE=1
  hostname: 5516454f8e45
  iterations_since_restore: 1
  mean_accuracy: -3.333333333333334
  node_ip: 172.28.0.2
  pid: 2987
  time_since_restore: 0.022668838500976562
  time_this_iter_s: 0.022668838500976562
  time_total_s: 0.022668838500976562
  timestamp: 1588899417
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: '00000'
  


Trial name,status,loc,N_GEN,POPULATION_SIZE,acc,iter,total time (s)
train_reward_function_00000,RUNNING,172.28.0.2:2987,1,1,-3.33333,0,0.0226688
train_reward_function_00001,RUNNING,,4,1,,,
train_reward_function_00002,PENDING,,1,3,,,
train_reward_function_00003,PENDING,,4,3,,,


(pid=2987) gen	nevals	avg     	std	min     	max     
(pid=2987) 0  	1     	-1.66667	0  	-1.66667	-1.66667
(pid=2987) 1  	0     	-1.66667	0  	-1.66667	-1.66667
(pid=2987) ----------test_reward -3.333333333333334
(pid=2986) 2020-05-08 00:56:57,302	INFO trainable.py:217 -- Getting current IP.
(pid=2986) gen	nevals	avg     	std	min     	max     
(pid=2986) 0  	1     	0.833333	0  	0.833333	0.833333
(pid=2986) 1  	0     	0.833333	0  	0.833333	0.833333
(pid=2986) 2  	0     	0.833333	0  	0.833333	0.833333
(pid=2986) 3  	0     	0.833333	0  	0.833333	0.833333
(pid=2986) 4  	1     	0.833333	0  	0.833333	0.833333
Result for train_reward_function_00001:
  date: 2020-05-08_00-56-57
  done: false
  experiment_id: 9ee68ae8b7084efc8bc24526ef36a8af
  experiment_tag: 1_N_GEN=4,POPULATION_SIZE=1
  hostname: 5516454f8e45
  iterations_since_restore: 1
  mean_accuracy: 1.6666666666666672
  node_ip: 172.28.0.2
  pid: 2986
  time_since_restore: 0.03384089469909668
  time_this_iter_s: 0.03384089469909668
  time

Trial name,status,loc,N_GEN,POPULATION_SIZE,acc,iter,total time (s)
train_reward_function_00000,TERMINATED,,1,1,-3.33333,0,0.0226688
train_reward_function_00001,TERMINATED,,4,1,1.66667,0,0.0338409
train_reward_function_00002,TERMINATED,,1,3,1.66667,0,0.054117
train_reward_function_00003,RUNNING,172.28.0.2:3067,4,3,1.66667,0,0.0750892


(pid=3067) 2  	2     	0.833333    	0          	0.833333	0.833333
(pid=3067) 3  	2     	0.833333    	0          	0.833333	0.833333
(pid=3067) 4  	2     	0.833333    	0          	0.833333	0.833333
(pid=3067) ----------test_reward 1.6666666666666672


Trial name,status,loc,N_GEN,POPULATION_SIZE,acc,iter,total time (s)
train_reward_function_00000,TERMINATED,,1,1,-3.33333,0,0.0226688
train_reward_function_00001,TERMINATED,,4,1,1.66667,0,0.0338409
train_reward_function_00002,TERMINATED,,1,3,1.66667,0,0.054117
train_reward_function_00003,TERMINATED,,4,3,1.66667,0,0.0750892


Best config:  {'N_GEN': 4, 'POPULATION_SIZE': 1}


In [10]:
print(evaluate_individual([0.9298461960519508, 0.7449587149229808, 0.4628576259710946, -0.282921307700329, -0.7019321146761455, 0.618488821337605])) #Ideal reward, altruistic agent

(0.833333333333333,)


In [11]:
print(evaluate_individual([1, 0, -1, 0, -1, 0])) #Worst reward, selfish agent

(-1.6666666666666667,)


In [12]:
print("Time Spent = ", (time.process_time() - start)/60, " minutes")

Time Spent =  0.014906510616666665  minutes
